In [1]:
using DataFrames
using CSV
using GLM
using Statistics

# Cargar el DataFrame (asumiendo que ya se ha realizado el EDA)
df = CSV.read("bottle_preprocesado.csv", DataFrame);

In [2]:
println(names(df))

["Cst_Cnt", "Btl_Cnt", "Sta_ID", "Depth_ID", "Depthm", "T_degC", "Salnty", "O2ml_L", "STheta", "O2Sat", "Oxy_µmol/Kg", "RecInd", "T_prec", "S_prec", "P_qual", "Chlqua", "Phaqua", "PO4q", "SiO3qu", "NO2q", "NO3q", "NH3q", "C14A1q", "C14A2q", "DarkAq", "MeanAq", "R_Depth", "R_TEMP", "R_POTEMP", "R_SALINITY", "R_SIGMA", "R_SVA", "R_DYNHT", "R_O2", "R_O2Sat", "R_PRES"]


In [3]:
# Obtener el índice de la columna "Oxy_µmol/Kg" 
# (Ajusta el nombre si es necesario)
oxy_index = findfirst(names(df) .== "Oxy_µmol/Kg") 

# Verificar si se encontró el índice
println(oxy_index)

11


In [4]:
# Crear una nueva columna con los valores de "Oxy_µmol/Kg"
df[!, :Oxy_µmol_Kg] = df[:, oxy_index]

864863-element Vector{Union{Missing, Float64}}:
    missing
    missing
    missing
    missing
    missing
    missing
    missing
    missing
    missing
    missing
    missing
    missing
    missing
   ⋮
  96.49
  86.56
  48.35
  47.67
  31.34
  19.83
  15.95
 253.47
 253.47
 253.06
 253.92
 252.08

In [5]:
# Definir las variables independientes a considerar (con el nuevo nombre)
independent_vars = [:Depthm, :Salnty, :O2ml_L, :STheta, :O2Sat, 
                    :Oxy_µmol_Kg, :R_Depth, :R_SALINITY, :R_SIGMA, :R_SVA, :R_DYNHT, 
                    :R_O2, :R_O2Sat, :R_PRES]

# Función para evaluar el modelo de regresión
function evaluate_model(df, formula)
    model = lm(formula, df)
    r2 = r²(model)
    println("Fórmula: ", formula)
    println("R²: ", r2)
    println(model)
    println("\n")
    return r2
end

evaluate_model (generic function with 1 method)

In [6]:
# Probar diferentes combinaciones de variables independientes
best_r2 = 0.0
best_formula = ""

# 1. Regresión con todas las variables
formula = @formula(T_degC ~ Depthm + Salnty + O2ml_L + STheta + O2Sat + 
                         Oxy_µmol_Kg + R_Depth + R_SALINITY + R_SIGMA + R_SVA + R_DYNHT + 
                         R_O2 + R_O2Sat + R_PRES)
r2 = evaluate_model(df, formula)
if r2 > best_r2
    best_r2 = r2
    best_formula = formula
end

# 2. Regresión con las variables más correlacionadas con T_degC
formula = @formula(T_degC ~ Depthm + O2ml_L + STheta + O2Sat + Oxy_µmol_Kg)
r2 = evaluate_model(df, formula)
if r2 > best_r2
    best_r2 = r2
    best_formula = formula
end

# 3. Regresión con otras combinaciones de variables (puedes agregar más)
# ...

# Mostrar la mejor combinación de variables
println("Mejor combinación de variables:")
println("Fórmula: ", best_formula)
println("R²: ", best_r2)

Fórmula: T_degC ~ Depthm + Salnty + O2ml_L + STheta + O2Sat + Oxy_μmol_Kg + R_Depth + R_SALINITY + R_SIGMA + R_SVA + R_DYNHT + R_O2 + R_O2Sat + R_PRES
R²: 0.9953717446431661
StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

T_degC ~ 1 + Depthm + Salnty + O2ml_L + STheta + O2Sat + Oxy_μmol_Kg + R_Depth + R_SALINITY + R_SIGMA + R_SVA + R_DYNHT + R_O2 + R_O2Sat + R_PRES

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                    Coef.   Std. Error        t  Pr(>|t|)    Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)  -507.179      1.62137      -312.81    <1e-99  -510.357     -504.001
Depthm          0.120741   0.143383        0.84    0.3997    -0.160285     0.401767
Salnty          8.84017    0.839311       10.53    <

In [6]:
# Eliminar filas con valores nulos en la columna objetivo ("T_degC")
# --- deleteRow(column) ---
# --- deleteRow(column) ---
function deleteRow(df::DataFrame, column::Symbol)
    filter!(!ismissing, df[:, column])
    return df
end
deleteRow(df, :T_degC)

864863×36 DataFrame
    Row │ Cst_Cnt  Btl_Cnt  Sta_ID       Depth_ID                           De ⋯
        │ Int64    Int64    String15     String                             In ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │       1        1  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…     ⋯
      2 │       1        2  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      3 │       1        3  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      4 │       1        4  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      5 │       1        5  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…     ⋯
      6 │       1        6  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      7 │       1        7  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      8 │       1        8  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
      9 │       1        9  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…     ⋯
     10 │       1       10  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
     11 │       1       11  054.0 056.0  19-4903CR-HY-060-0930-05400560-0…
   ⋮    │    ⋮        ⋮          ⋮                       ⋮                     ⋱
 864854 │   34403   864854  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864855 │   34403   864855  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…     ⋯
 864856 │   34403   864856  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864857 │   34403   864857  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864858 │   34403   864858  093.3 120.0  20-1611SR-MX-313-2053-09331200-0…
 864859 │   34404   864859  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…     ⋯
 864860 │   34404   864860  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…
 864861 │   34404   864861  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…
 864862 │   34404   864862  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…
 864863 │   34404   864863  093.4 026.4  20-1611SR-MX-310-2239-09340264-0…     ⋯
                                              32 columns and 864842 rows omitted

In [7]:
# Seleccionar solo las columnas numéricas
 numeric_cols = names(df, findall(eltype.(eachcol(df)) .<: Union{Missing, Number}))
 df_numeric = df[:, numeric_cols]
 
# Eliminar filas con missings en las columnas numéricas
 df_numeric = dropmissing(df_numeric)


253439×34 DataFrame
    Row │ Cst_Cnt  Btl_Cnt  Depthm  T_degC   Salnty   O2ml_L   STheta   O2Sat  ⋯
        │ Int64    Int64    Int64   Float64  Float64  Float64  Float64  Float6 ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │      71     2161       0    10.3    33.03      5.9    25.364     92. ⋯
      2 │      71     2163      10    10.29   32.951     6.04   25.304     95.
      3 │      71     2164      15    10.29   32.99      6.06   25.335     95.
      4 │      71     2165      20    10.33   33.005     6.04   25.339     95.
      5 │      71     2166      30    10.39   33.054     6.01   25.368     94. ⋯
      6 │      71     2167      31    10.4    33.06      6.01   25.371     94.
      7 │      71     2168      47    10.36   33.15      5.78   25.448     91.
      8 │      71     2169      50    10.35   33.167     5.76   25.463     90.
      9 │      71     2170      64    10.25   33.26      5.6    25.553     88. ⋯
     10 │      71     2171      75    10.05   33.381     5.07   25.681     79.
     11 │      71     2172      98     9.54   33.64      3.82   25.969     59.
   ⋮    │    ⋮        ⋮       ⋮        ⋮        ⋮        ⋮        ⋮        ⋮   ⋱
 253430 │   32581   817793     250     7.75   34.115     1.47   26.619     22.
 253431 │   32581   817795     300     7.44   34.165     1.01   26.704     15. ⋯
 253432 │   32581   817798     400     6.65   34.212     0.59   26.851      8.
 253433 │   32581   817800     500     5.93   34.26      0.4    26.982      5.
 253434 │   32582   817822     200     8.38   34.058     1.75   26.48      26.
 253435 │   32582   817824     250     7.83   34.088     1.51   26.587     22. ⋯
 253436 │   32582   817826     300     7.09   34.115     1.18   26.713     17.
 253437 │   32582   817829     400     6.5    34.196     0.62   26.858      9.
 253438 │   32582   817831     500     5.84   34.252     0.4    26.987      5.
 253439 │   33073   830773       0    18.05   33.586     6.4    24.178    118. ⋯
                                              27 columns and 253418 rows omitted